### Импорт библиотек

In [73]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import SimpleImputer

### Загрузка данных. Анализ.

In [74]:
data = pd.read_csv('ranking.csv')

## Описание датасета
Осу! это музыкальная ритм-игра с 4 режимами. В этом наборе данных вы можете просмотреть рейтинг в стандартном режиме, сделанном 26.03.2017 около 12 вечера. Ранжирование основывается на pp (очках производительности), присуждаемых после каждой игры, на которые влияет точность игры и оценка; pps затем суммируются с весами: ваша лучшая игра наградит вас всеми pp очками на карте, затем процент уменьшится (это может поддерживать баланс между сильными игроками и игроками, которые играют слишком много). Вы можете найти здесь много других статистических данных.

### Содержание
Набор данных содержит несколько столбцов, в которых представлена статистика по каждому игроку в топ-100 игры в стандартном режиме. Рейтинг упорядочен по pp. Некоторые игроки, похоже, имеют одинаковые очки, но есть десятичные дроби, которые не отображаются в таблице рейтинга на сайте

In [75]:
data.head()

,rank,country_rank,player_name,country,accuracy,play_count,level,hours,performance_points,ranked_score,ss,s,a,watched_by,total_hits,device
0,1,1,Cookiezi,South Korea,98.86,15245.0,101,324,13450,22401473154,55,454,325,3057421,6372761,tablet
1,2,2,Angelsim,South Korea,98.87,124726.0,103,1457,12950,35481061465,159,1369,679,498548,29845228,mouse
2,3,1,Rafis,Poland,99.03,NaN,103,2229,12942,36343320002,170,1654,785,1170973,35760655,tablet
3,4,1,Vaxei,United States,98.03,93012.0,101,1034,12735,18096595619,34,541,563,152551,15601281,tablet
4,5,1,hvick225,Taiwan,98.93,134052.0,102,1970,12421,47072997400,826,3466,701,2015824,22576332,tablet


In [76]:
data.shape

(100, 16)

In [77]:
data.dtypes

rank                    int64
country_rank            int64
player_name            object
country                object
accuracy              float64
play_count            float64
level                   int64
hours                   int64
performance_points      int64
ranked_score            int64
ss                      int64
s                       int64
a                       int64
watched_by              int64
total_hits              int64
device                 object
dtype: object

In [78]:
data.isnull().sum()

rank                  0
country_rank          0
player_name           0
country               0
accuracy              0
play_count            3
level                 0
hours                 0
performance_points    0
ranked_score          0
ss                    0
s                     0
a                     0
watched_by            0
total_hits            0
device                0
dtype: int64

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rank                100 non-null    int64  
 1   country_rank        100 non-null    int64  
 2   player_name         100 non-null    object 
 3   country             100 non-null    object 
 4   accuracy            100 non-null    float64
 5   play_count          97 non-null     float64
 6   level               100 non-null    int64  
 7   hours               100 non-null    int64  
 8   performance_points  100 non-null    int64  
 9   ranked_score        100 non-null    int64  
 10  ss                  100 non-null    int64  
 11  s                   100 non-null    int64  
 12  a                   100 non-null    int64  
 13  watched_by          100 non-null    int64  
 14  total_hits          100 non-null    int64  
 15  device              100 non-null    object 
dtypes: float6

## Обработка пропусков

In [80]:
# Удаляем ненужные колонны 
data.drop(['watched_by','level','player_name','country'], axis = 1, inplace = True)

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rank                100 non-null    int64  
 1   country_rank        100 non-null    int64  
 2   accuracy            100 non-null    float64
 3   play_count          97 non-null     float64
 4   hours               100 non-null    int64  
 5   performance_points  100 non-null    int64  
 6   ranked_score        100 non-null    int64  
 7   ss                  100 non-null    int64  
 8   s                   100 non-null    int64  
 9   a                   100 non-null    int64  
 10  total_hits          100 non-null    int64  
 11  device              100 non-null    object 
dtypes: float64(2), int64(9), object(1)
memory usage: 9.5+ KB


In [82]:
# Заполняем отсутствующие значения время проведённого в игре, средним значением этого параметра
data['play_count'] = data['play_count'].replace(0,np.nan)
data['play_count'] = data['play_count'].fillna(data['play_count'].mean())

In [83]:
data.isnull().sum()

rank                  0
country_rank          0
accuracy              0
play_count            0
hours                 0
performance_points    0
ranked_score          0
ss                    0
s                     0
a                     0
total_hits            0
device                0
dtype: int64

## Преобразование категориальных признаков в числовые

In [84]:
data['device'].value_counts()

tablet       87
mouse        10
not_known     2
touch         1
Name: device, dtype: int64

In [85]:
# Девайс кодируем в 0/1/2/3
data['Device']=data.device.replace({'tablet':0,'mouse':1,'not_known':2,'touch':3})
data.drop('device', axis = 1, inplace = True)

In [86]:
data.head(pd.set_option('display.max_rows', None))

,rank,country_rank,accuracy,play_count,hours,performance_points,ranked_score,ss,s,a,total_hits,Device
0,1,1,98.86,15245.000000,324,13450,22401473154,55,454,325,6372761,0
1,2,2,98.87,124726.000000,1457,12950,35481061465,159,1369,679,29845228,1
2,3,1,99.03,98281.670103,2229,12942,36343320002,170,1654,785,35760655,0
3,4,1,98.03,93012.000000,1034,12735,18096595619,34,541,563,15601281,0
4,5,1,98.93,134052.000000,1970,12421,47072997400,826,3466,701,22576332,0
5,6,1,98.59,114629.000000,1476,12211,29533435386,196,1516,585,24265869,0
6,7,1,98.74,98281.670103,844,11314,38146854078,426,1503,476,18502111,0
7,8,1,98.89,83839.000000,1246,11208,39126859229,112,2268,1336,19686899,0
8,9,1,99.25,98281.670103,805,11105,33756034097,308,1205,864,11745064,1
9,10,2,99.23,78862.000000,1058,11054,21973937247,29,658,716,22604434,0


## Масштабирование значений

In [87]:
from sklearn.preprocessing import StandardScaler

In [88]:
sc2 = StandardScaler()
sc2.fit(data)
sc2_data = sc2.transform(data)

In [89]:
data = pd.DataFrame(sc2_data, columns = data.columns)

In [91]:
data.head(pd.set_option('display.max_rows', None))

,rank,country_rank,accuracy,play_count,hours,performance_points,ranked_score,ss,s,a,total_hits,Device
0,-1.714816,-0.903214,0.199143,-2.158703e+00,-1.813137,3.807082,-0.670784,-0.307313,-1.011878,-1.334763,-1.967118,-0.346219
1,-1.680173,-0.597040,0.216490,6.874727e-01,0.328549,3.234232,0.087629,-0.175582,-0.204845,-0.385593,1.148233,1.690361
2,-1.645531,-0.903214,0.494042,-3.783058e-16,1.787845,3.225066,0.137626,-0.161649,0.046526,-0.101379,1.933350,-0.346219
3,-1.610888,-0.903214,-1.240656,-1.369955e-01,-0.471039,2.987906,-0.920400,-0.333912,-0.935144,-0.696621,-0.742275,-0.346219
4,-1.576245,-0.903214,0.320572,9.299205e-01,1.298263,2.628156,0.759781,0.669268,1.644715,-0.326605,0.183479,-0.346219
5,-1.541602,-0.903214,-0.269225,4.249812e-01,0.364465,2.387559,-0.257241,-0.128716,-0.075191,-0.637633,0.407720,-0.346219
6,-1.506960,-0.903214,-0.009020,-3.783058e-16,-0.830192,1.359866,0.242203,0.162611,-0.086657,-0.929891,-0.357267,-0.346219
7,-1.472317,-0.903214,0.251184,-3.754658e-01,-0.070299,1.238422,0.299028,-0.235114,0.588075,1.376001,-0.200017,-0.346219
8,-1.437674,-0.903214,0.875675,-3.783058e-16,-0.903913,1.120415,-0.012396,0.013148,-0.349494,0.110441,-1.254087,1.690361
9,-1.403031,-0.597040,0.840981,-5.048528e-01,-0.425672,1.061984,-0.695574,-0.340245,-0.831949,-0.286386,0.187208,-0.346219
